In [ ]:
%matplotlib inline

In [ ]:
from pathlib import Path

from spectral_cube import SpectralCube
from astropy.stats import mad_std

In [ ]:
# data_path = Path("/reduction10/erickoch/LGLBS/line_imaging/psf_and_noise_tests/")
data_path = Path("/reduction/erickoch/LGLBS/line_imaging/psf_and_noise_tests/")

In [ ]:
configs = ['A', 'B', 'C', 'D']

config_combos = ['A+B+C+D', 'A+B+C', 'B+C+D', 'C+D']

briggs_weights = [f"briggs_{this_r}" for this_r in
                  [2, 1.5, 1.0, 0.5, 0.0, -0.5, -1.0, -1.5, -2.0]]
weightings = ['natural'] + briggs_weights + ['uniform']

In [ ]:
# ic10ctr_A_hi21cm_1p2kms_briggs_-2.0.image

this_field = 'ic10ctr'
this_line = "hi21cm_1p2kms"

beams = {}
rms = {}
rms_K = {}

hi_freq = 1.42040575177 * u.GHz

use_circ_beam = True

for this_config in configs + config_combos:

    beams[this_config] = {}
    rms[this_config] = {}
    rms_K[this_config] = {}

    for this_weight in weightings:

        this_imagename = f"{this_field}_{this_config}_{this_line}_{this_weight}"

        if not (data_path / f"{this_imagename}.image").exists():
            print(f"No image exists: {this_imagename}")
            continue
        this_cube = SpectralCube.read(data_path / f"{this_imagename}.image", format='casa')
        this_cube = this_cube.with_spectral_unit(u.km / u.s, 'radio')

        print(this_cube.header['CDELT3'])

        this_proj = this_cube[0]

        # Account for change in beam using the ratio of the areas
        if use_circ_beam:
            beam_area_ratio = (this_proj.beam.sr / Beam(this_proj.beam.major).sr).value
        else:
            beam_area_ratio = 1.

        beams[this_config][this_weight] = this_proj.beam
        rms[this_config][this_weight] = mad_std(this_proj.value, ignore_nan=True)
        rms_K[this_config][this_weight] = mad_std(this_proj.to(u.K).value * beam_area_ratio, ignore_nan=True)

chan_width = np.abs(this_cube.header['CDELT3']) * u.km / u.s

In [ ]:
beams

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sb

sb.set_context("paper", font_scale=1.2)

In [ ]:
# plt.figure(figsize=(3.2, 3.2))
# plt.figure(figsize=(3.2, 3.2))

fig, axs = plt.subplots(1, 2, sharex=True, figsize=(6.4, 3.2),
                        layout='constrained')


for this_config in configs + config_combos:

    these_major_arcsec = [beams[this_config][this_weight].major.to(u.arcsec).value
                          for this_weight in weightings]
    these_rms = np.array([rms[this_config][this_weight]
                 for this_weight in weightings])
    these_rms_K = np.array([rms_K[this_config][this_weight]
                 for this_weight in weightings])
    axs[0].loglog(these_major_arcsec, these_rms, "D-", label=this_config)

    axs[1].loglog(these_major_arcsec, these_rms_K, "D-", label=this_config)

    print(these_rms.min(), these_rms_K.min())

axs[1].legend(fontsize=5)

axs[0].set_ylabel("rms (Jy/beam)")
axs[1].set_ylabel("rms (K)")

axs[0].set_xlabel("Beam Major Axis ($^{''}$)")
axs[1].set_xlabel("Beam Major Axis ($^{''}$)")

axs[0].set_xlim([1, 70])

axs[0].axvline(60, color='k')
axs[1].axvline(60, color='k')

axs[1].axvline(4, color='k')


In [ ]:
# plt.figure(figsize=(3.2, 3.2))
# plt.figure(figsize=(3.2, 3.2))

fig, axs = plt.subplots(1, 2, sharex=True, figsize=(6.4, 3.2),
                        layout='constrained')


lines = []
labels = []

rep_vel = 10 # * km/s

for this_config in configs + config_combos:

    these_major_arcsec = [beams[this_config][this_weight].major.to(u.arcsec).value
                          for this_weight in weightings]
    these_rms_K = np.array([rms_K[this_config][this_weight]
                            for this_weight in weightings])

    rms_per0p4 = these_rms_K * np.sqrt(chan_width.value / 0.42)

    lines.append(axs[0].loglog(these_major_arcsec, rms_per0p4, "D-", label=this_config)[0])
    labels.append(this_config)

    rms_10kms = these_rms_K / np.sqrt(rep_vel / chan_width)
    coldens = 1.82e18 * rms_10kms * rep_vel
    axs[1].loglog(these_major_arcsec, coldens,
                  "D-", label=this_config)

    # print(these_rms.min(), these_rms_K.min())

# Show the scaling from simply smoothing
scaling_beams = np.logspace(0, 2, 10)

for noise_scale in np.logspace(np.log10(100), np.log10(5000), 5):
    axs[0].loglog(scaling_beams, noise_scale / scaling_beams**2, linestyle='--', color='k', alpha=0.5)

    rms_10kms = noise_scale / np.sqrt(rep_vel / chan_width)
    coldens = 1.82e18 * rms_10kms * rep_vel

    axs[1].loglog(scaling_beams,
                  coldens / scaling_beams**2,
                  linestyle='--', color='k', alpha=0.5)


# Move to outside the panels
# axs[0].legend(fontsize=10, ncol=2)

# axs[0].set_ylabel(f"HI rms (K) per {chan_width.value:.1f}" + r"km s$^{-1}$")
axs[0].set_ylabel("HI rms (K) per 0.4" + r" km s$^{-1}$")

axs[1].set_ylabel("HI rms column density (cm$^{-2}$)")
# over 10 km s$^{-1}$", nrow=2)
axs[1].text(0.95, 0.9, 'per 10 km s$^{-1}$',
        transform=axs[1].transAxes,
        horizontalalignment='right',
        verticalalignment='center',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white', linewidth=1))

axs[0].set_xlabel("Beam Major Axis ($^{''}$)")
axs[1].set_xlabel("Beam Major Axis ($^{''}$)")

axs[0].set_xlim([1, 70])

axs[0].set_ylim([0.5, 800])
axs[1].set_ylim([1e18, 5e21])

# axs[1].tick_params(labeltop=True, top=True)
axs[1].xaxis.set_tick_params(which='both', top=True)
axs[1].yaxis.set_tick_params(which='both', right=True)

# axs[0].axvline(60, color='k')
# axs[1].axvline(60, color='k')

# High-res target is 1e20 @ 7 km/s. Convert to equivalent over 10 km/s
# axs[1].axvline(4.5, color='k')
# axs[1].axhline(1e20 * (7 / 10), color='k')

axs[1].plot([4.5] * 2, [1e20 * (7 / 10), 1e22], 'k')
axs[1].plot([1, 4.5], [1e20 * (7 / 10)] * 2, 'k')
axs[1].text(1.1, 5e19, r"$7\times10^{19}$ cm$^{-2}$", va='top', ha='left')
axs[1].text(4.5, 1e22, r"$4.5^{''}$", va='center', ha='center', rotation=0)


# High-res target is 1e20 @ 7 km/s. Convert to equivalent over 10 km/s
# axs[1].axvline(48.6, color='k')
# axs[1].axhline(3.2e18 * (20 / 10), color='k')

axs[1].plot([48.6] * 2, [3.2e18 * (20 / 10), 1e22], 'k')
axs[1].plot([1, 48.6], [3.2e18 * (20 / 10)] * 2, 'k')
axs[1].text(1.1, 5e18, r"$6\times10^{18}$ cm$^{-2}$", va='top', ha='left')
axs[1].text(48.6, 1e22, r"$48.6^{''}$", va='center', ha='center', rotation=0)


fig.legend(lines, labels,
           fontsize=10, ncol=4,
           bbox_to_anchor=(0.85, 1.2),)

# Adjust figure to make room for the legend
# plt.subplots_adjust(right=0.85)

# plt.savefig("figures/ic10ctr_beam_vs_noise.pdf", bbox_inches='tight')


In [ ]:
1e20 * (7 / 10)

### HI mass sensitivity

Mass sensitivity per beam calcs.

In [ ]:
def beam_phys_size(beam, distance):
    return beam.sr.to(u.rad**2).value * distance.to(u.pc)**2

In [ ]:
distance = 790 * u.kpc

fig, axs = plt.subplots(1, 3, sharex=True, figsize=(6.4, 3.2),
                        layout='constrained')


lines = []
labels = []

rep_vel = 10 # * km/s

for this_config in configs + config_combos:

    these_major_arcsec = [beams[this_config][this_weight].major.to(u.arcsec).value
                          for this_weight in weightings]

    these_beam_areas = np.array([beam_phys_size(beams[this_config][this_weight], distance).to(u.pc**2).value
                                 for this_weight in weightings]) * u.pc**2
    these_rms_K = np.array([rms_K[this_config][this_weight]
                            for this_weight in weightings])

    rms_per0p4 = these_rms_K * np.sqrt(chan_width.value / 0.42)

    lines.append(axs[0].loglog(these_major_arcsec, rms_per0p4, "D-", label=this_config)[0])
    labels.append(this_config)

    rms_10kms = these_rms_K / np.sqrt(rep_vel / chan_width)
    coldens = 1.82e18 * rms_10kms.value * rep_vel * u.cm**-2
    axs[1].loglog(these_major_arcsec, coldens,
                  "D-", label=this_config)


    hi_mass_sensitivity = c.m_p.to(u.solMass) * (coldens * these_beam_areas).to(u.one)
    axs[2].loglog(these_major_arcsec, hi_mass_sensitivity.value,
                  "D-", label=this_config)

scaling_beams = np.logspace(0, 2, 10)

FWHM_TO_AREA = 2*np.pi/(8*np.log(2))

for noise_scale in np.logspace(np.log10(100), np.log10(5000), 5):
    axs[0].loglog(scaling_beams, noise_scale / scaling_beams**2, linestyle='--', color='k', alpha=0.5)

    rms_10kms = noise_scale / np.sqrt(rep_vel / chan_width)
    coldens = 1.82e18 * rms_10kms.value * rep_vel * u.cm**-2

    axs[1].loglog(scaling_beams,
                  coldens / scaling_beams**2,
                  linestyle='--', color='k', alpha=0.5)

    phys_scaling_beams = (scaling_beams * u.arcsec).to(u.rad).value * distance.to(u.pc)

    # HI mass sensitivity
    # axs[2].loglog(scaling_beams,
    #               c.m_p.to(u.solMass) * (coldens * FWHM_TO_AREA * phys_scaling_beams**2).to(u.one),
    #               linestyle='--', color='k', alpha=0.5)

In [ ]:
FWHM_TO_AREA * phys_scaling_beams**2

In [ ]:
these_beam_areas

In [ ]:

fig, axs = plt.subplots(1, 3, sharex=True, figsize=(8.4, 3.2),
                        layout='constrained')


lines = []
labels = []

rep_vel = 10 # * km/s

markers = ['o', 's', 'd', 'D', 'H', 'X', 'p', 'h']

colors = sb.color_palette("colorblind", n_colors=len(config_combos + configs))[::-1]

for this_config, this_marker, this_color in zip(configs[::-1] + config_combos[::-1], markers, colors):

    these_major_arcsec = [beams[this_config][this_weight].major.to(u.arcsec).value
                          for this_weight in weightings]

    these_beam_areas = np.array([beam_phys_size(beams[this_config][this_weight], distance).to(u.pc**2).value
                                 for this_weight in weightings]) * u.pc**2
    these_rms_K = np.array([rms_K[this_config][this_weight]
                            for this_weight in weightings])

    rms_per0p4 = these_rms_K * np.sqrt(chan_width.value / 0.42)

    lines.append(axs[0].loglog(these_major_arcsec, rms_per0p4, f"{this_marker}-", label=this_config,
                               markersize=8, markeredgecolor='k', markeredgewidth=0.1, alpha=0.8,
                               color=this_color)[0])
    labels.append(this_config)

    rms_10kms = these_rms_K / np.sqrt(rep_vel / chan_width)
    coldens = 1.82e18 * rms_10kms.value * rep_vel * u.cm**-2
    axs[1].loglog(these_major_arcsec, coldens,
                  f"{this_marker}-", label=this_config,
                  markersize=8, markeredgecolor='k', markeredgewidth=0.1, alpha=0.8,
                  color=this_color)


    hi_mass_sensitivity = c.m_p.to(u.solMass) * (coldens * these_beam_areas).to(u.one)
    axs[2].loglog(these_major_arcsec, hi_mass_sensitivity.value,
                  f"{this_marker}-", label=this_config,
                  markersize=8, markeredgecolor='k', markeredgewidth=0.1, alpha=0.8,
                  color=this_color)


# Show the scaling from simply smoothing
scaling_beams = np.logspace(0, 2, 10)

for noise_scale in np.logspace(np.log10(100), np.log10(5000), 5):
    axs[0].loglog(scaling_beams, noise_scale / scaling_beams**2,
                  linestyle='--', color='k', alpha=0.25, linewidth=0.5, markersize=8,
                  zorder=-1)

    rms_10kms = noise_scale / np.sqrt(rep_vel / chan_width)
    coldens = 1.82e18 * rms_10kms * rep_vel

    axs[1].loglog(scaling_beams,
                  coldens / scaling_beams**2,
                  linestyle='--', color='k', alpha=0.25, linewidth=0.5,
                  zorder=-1)


# Move to outside the panels
# axs[0].legend(fontsize=10, ncol=2)

# axs[0].set_ylabel(f"HI rms (K) per {chan_width.value:.1f}" + r"km s$^{-1}$")
# axs[0].set_ylabel("HI rms (K) per 0.4" + r" km s$^{-1}$")
axs[0].set_ylabel("HI rms (K) per 0.4 km s$^{-1}$")

# axs[0].text(0.95, 0.9, 'per 0.4 km s$^{-1}$',
#         transform=axs[0].transAxes,
#         horizontalalignment='right',
#         verticalalignment='center',
#         bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white', linewidth=1))

axs[1].set_ylabel("HI rms column density (cm$^{-2}$)")
# over 10 km s$^{-1}$", nrow=2)
axs[1].text(0.95, 0.9, 'per 10 km s$^{-1}$',
        transform=axs[1].transAxes,
        horizontalalignment='right',
        verticalalignment='center',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white', linewidth=1))

axs[2].set_ylabel("M$_\mathrm{HI}$ (M$_{\odot}$) per 10 km s$^{-1}$")
# axs[2].text(0.95, 0.9, 'per 10 km s$^{-1}$',
#         transform=axs[2].transAxes,
#         horizontalalignment='right',
#         verticalalignment='center',
#         bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white', linewidth=1))

axs[0].set_xlabel("Beam Major Axis ($^{''}$)")
axs[1].set_xlabel("Beam Major Axis ($^{''}$)")
axs[2].set_xlabel("Beam Major Axis ($^{''}$)")

axs[0].set_xlim([1, 70])

axs[0].set_ylim([0.5, 800])
axs[1].set_ylim([1e18, 5e21])
axs[2].set_ylim([1e2, 5e3])

# axs[1].tick_params(labeltop=True, top=True)
axs[1].xaxis.set_tick_params(which='both', top=True)
axs[1].yaxis.set_tick_params(which='both', right=True)

# axs[0].axvline(60, color='k')
# axs[1].axvline(60, color='k')

# High-res target is 1e20 @ 7 km/s. Convert to equivalent over 10 km/s
# axs[1].axvline(4.5, color='k')
# axs[1].axhline(1e20 * (7 / 10), color='k')

axs[1].plot([4.5] * 2, [1e20 * (7 / 10), 1e22], 'k', zorder=2, alpha=0.25, linewidth=5)
axs[1].plot([1, 4.5], [1e20 * (7 / 10)] * 2, 'k', zorder=2, alpha=0.25, linewidth=5)
# axs[1].text(1.1, 5e19, r"$7\times10^{19}$ cm$^{-2}$", va='top', ha='left')
axs[1].text(4.5, 1e22, r"$4.5^{''}$", va='center', ha='center', rotation=0)


# High-res target is 1e20 @ 7 km/s. Convert to equivalent over 10 km/s
# axs[1].axvline(48.6, color='k')
# axs[1].axhline(3.2e18 * (20 / 10), color='k')

axs[1].plot([48.6] * 2, [3.2e18 * (20 / 10), 1e22], 'k', zorder=2, alpha=0.25, linewidth=5)
axs[1].plot([1, 48.6], [3.2e18 * (20 / 10)] * 2, 'k', zorder=2, alpha=0.25, linewidth=5)
# axs[1].text(1.1, 5e18, r"$6\times10^{18}$ cm$^{-2}$", va='top', ha='left')
axs[1].text(48.6, 1e22, r"$48.6^{''}$", va='center', ha='center', rotation=0)


# Manually give the equivalent rms per 10 km/s
scale_factor = np.sqrt(10 / 0.4)
ax2 = axs[0].twinx()
yticks_ax1 = axs[0].get_yticks()
ax2.set_yscale('log')

# ax2.set_yticks([val/scale_factor for val in yticks_ax1])
ax2.set_ylim([0.5 / scale_factor, 800 / scale_factor])
ax2.set_yticks([1.e-2, 1e-1, 1, 1e1, 1e2])

print([yticks_ax1[0] / scale_factor, yticks_ax1[-1] / scale_factor])
print([yticks_ax1[0], yticks_ax1[-1]])

# ax2.set_yticklabels([f"{val/scale_factor:.1f}" for val in yticks_ax1])

ax2.set_ylabel("HI rms (K) per 10 km s$^{-1}$")


fig.legend(lines, labels,
           fontsize=11, ncol=4,
           bbox_to_anchor=(0.82, 1.2),)

# Adjust figure to make room for the legend
# plt.subplots_adjust(right=0.85)

plt.savefig("figures/ic10ctr_beam_vs_noise.pdf", bbox_inches='tight')
